<a href="https://colab.research.google.com/github/pranavvp16/AnimeOnly/blob/main/Model_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#data_frame = pd.read_csv("drive/MyDrive/animeflv.csv")
data_frame = pd.read_scv("data/animeflv.csv")

In [ ]:
data_frame.info()

In [ ]:
tans=data_frame['description'][0]

In [ ]:
data_frame.dropna(inplace=True)

In [ ]:
#data_frame['description'].head()

In [ ]:
pip install googletrans==3.1.0a0

In [ ]:
import googletrans
translator = googletrans.Translator()
df1 = data_frame['description'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))
#print(df1)

In [ ]:
data_frame["description"] = df1

In [ ]:
data_frame["genders"] = data_frame['genders'].apply(translator.translate, src='es', dest='en').apply(getattr, args=('text',))

In [ ]:
data_frame.rename(columns ={'genders':'genres'},inplace = True)

In [ ]:
#taking data for training set 
anime_df = data_frame[["title","description","genres","image"]]

In [ ]:
anime_df.head()

In [ ]:
anime_df.drop_duplicates(inplace = True)

In [ ]:
anime_df["genres"].value_counts()

In [ ]:
anime_df["image"].value_counts()

In [ ]:
#import urllib.request

#get_url= urllib.request.urlopen('https://www3.animeflv.net/uploads/animes/covers/3379.jpg')

#print("Response Status: "+ str(get_url.getcode()) )

In [ ]:
anime_df["description"] = [i.replace("'","")for i in anime_df["description"]]
anime_df["description"] = [i.replace(",","")for i in anime_df["description"]]
anime_df["description"] = [i.replace('-',"")for i in anime_df["description"]]
anime_df["title"] = [i.replace("Pelicula","Movie")for i in anime_df["title"]]

In [ ]:
anime_df["title"].head()

In [ ]:
anime_df["description"]=anime_df["description"].apply(lambda x:x.split())
anime_df["genres"]=anime_df["genres"].apply(lambda x:x.split())

In [ ]:
anime_df['description'].head()

In [ ]:
anime_df['genres'].head()

In [ ]:
#anime_df['title'] = data_frame['title']

In [ ]:
anime_df["tags"] = anime_df['description'] + anime_df['genres']

In [ ]:
anime_df.isna().sum()

In [ ]:
#multi_chars = "[']"
#def convert(lst , sym): 
 #for i in lst:
  #temp_string = i
  #for ch in sym:
    #temp_string = temp_string.replace(ch,"")
    #for j in temp_string:
      #dta = []
      #dta.append(j)
 #return dta 

In [ ]:
anime_df["tags"]=anime_df["tags"].apply(lambda x:[i.replace(" ","")for i in x])
anime_df["tags"] = anime_df["tags"].apply(lambda x:" ".join(x))

In [ ]:
anime_df.head()

In [ ]:
anime_df=anime_df.reset_index(drop=True)

In [ ]:
animef_df = anime_df[["title","tags"]]

In [ ]:
animef_df.head()

In [ ]:
animef_df["tags"] = anime_df["tags"].apply(lambda x:x.lower())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=4000,stop_words="english")
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()

In [ ]:
ps.stem("abilities")

In [ ]:
def stem(text):
  y=[]

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y) 

In [ ]:
animef_df["tags"][1]

In [ ]:
stem(animef_df["tags"][1])

In [ ]:
animef_df["tags"]=animef_df["tags"].apply(stem)

In [ ]:
vectors = cv.fit_transform(animef_df["tags"]).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])

In [ ]:
def recommend(anime):
  anime_index = animef_df[animef_df['title']==anime].index[0]
  distances = similarity[anime_index]
  anime_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in anime_list:
    print(animef_df.iloc[i[0]].title)

In [ ]:
recommend('Fushigi no Umi no Nadia')

In [ ]:
import pickle 
pickle.dump(animef_df.to_dict(),open('anime_dict.pkl','wb'))
pickle.dump(animef_df,open('anime.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
pickle.dump(image_urls,open('image_urls.pkl','wb'))

In [ ]:
image_urls[0]